# Home work no.5
>## professor : Dr. Vahid Khazaei nezhad
# Team Members :
- Saeed Tizkar
- Pooriya Amini
- Mohammad Elahi

## فایل مربوط به online retail درون پوشه دیتاست قرار ندادم چون حجم زیادی داشت و آپلود زمانبر میشد

# Home work no.1
>## professor : Dr. Vahid Khazaei nezhad
# Team Members :
- Saeed Tizkar
- Pooriya Amini
- Mohammad Elahi# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

print('Libraries loaded.')

Libraries loaded.


# Example 1: Simple use of algorithm

## Creating Dataset

In [2]:
basket_lists = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
                ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
                ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
                ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
                ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

trans_encoder = TransactionEncoder()

encoded_baskets = trans_encoder.fit(basket_lists).transform(basket_lists)

dataset_df = pd.DataFrame(encoded_baskets, columns= trans_encoder.columns_)

dataset_df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


## Computing support values

In [3]:
apriori(dataset_df, min_support= 0.6, use_colnames= True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Kidney Beans, Milk)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Kidney Beans, Yogurt)"


# Example2: Association rule mining

## Loading dataset

In [4]:
basket_df = pd.read_excel('./Online Retail.xlsx')

basket_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Dataset description

## Some basic exploratory

In [5]:
basket_df.shape

(541909, 8)

In [6]:
basket_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
basket_df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
canceled = basket_df.loc[basket_df['InvoiceNo'].str.contains('C') == True]

canceled.shape

(9288, 8)

## Cleansing dataset

In [9]:
basket_df['Description'] = basket_df['Description'].str.strip()

#removing canceled records
basket_df.drop(canceled.index, inplace= True)

print(f'{canceled.shape[0]} has removed from dataset, new dataset size is {basket_df.shape[0]}')

9288 has removed from dataset, new dataset size is 532621


In [10]:
basket_df.loc[basket_df["Description"] == "x"]["Description"].empty

True

## Preparing dataset for analysing

### Selecting one region (Germany)

In [11]:
france_basket = basket_df.loc[basket_df['Country'] == 'France']

france_basket.shape

(8408, 8)

### Creating transaction, items set

In [12]:
def trans_basket_provider(in_basket):
    basket = []
    transaction_nom = []

    for transaction, items in in_basket.groupby('InvoiceNo').groups.items():

        products = []

        transaction_nom.append(transaction)
        for item in items:
            products.append(str(in_basket.loc[item]['StockCode']))        

        ###   
        basket.append(products)
    return (basket, transaction_nom)
fr_basket, fr_trans_nom = trans_basket_provider(france_basket)

### creating binary dataset

In [13]:
def binary_df_provider(in_basket, trans_nom) :
    trans_encoder = TransactionEncoder()

    encoded_baskets = trans_encoder.fit(in_basket).transform(in_basket)
    binary_df = pd.DataFrame(encoded_baskets, columns= trans_encoder.columns_, index= trans_nom)

    return binary_df
fr_binary_df = binary_df_provider(fr_basket, fr_trans_nom)
fr_binary_df

,10002,10120,10125,10135,11001,15036,15039,15044C,15056BL,15056N,...,90030C,90031,90099,90184B,90184C,90201B,90201C,C2,M,POST
536370,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
536852,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
536974,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
537065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
537463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
581001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
581171,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
581279,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Association extraction(Recommendation based on antecedents)

In [14]:
frequencies_items = apriori(fr_binary_df, min_support= 0.05, use_colnames= True)
frequencies_items
rules = association_rules(frequencies_items, metric= "lift", min_threshold= 1)

rules = rules.sort_values(['confidence', 'lift'], ascending= [False, False])

rules.reset_index(inplace= True, drop= True)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(20712),(POST),0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf
1,"(21731, 22554)",(POST),0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf
2,"(21731, 22556)",(POST),0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf
3,"(21086, 21080)",(21094),0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959
4,"(21080, 21094)",(21086),0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796


### Filtering results

In [15]:
condition0 = rules.antecedents.apply(lambda x: False if 'POST' in x else True)

condition1 = rules.consequents.apply(lambda x: False if 'POST' in x else True)

condition2 = rules['antecedents'].apply(lambda x: len(x) >= 2)

rules.loc[condition0 & condition1 & condition2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,"(21086, 21080)",(21094),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
4,"(21080, 21094)",(21086),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
53,"(22726, 22728)",(22727),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
54,"(22727, 22728)",(22726),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194
81,"(21086, 21094)",(21080),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
84,"(22727, 22726)",(22728),0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456
109,"(22551, 22556)",(22554),0.089286,0.170918,0.068878,0.771429,4.513433,0.053617,3.627232
143,"(22554, 22556)",(22551),0.102041,0.137755,0.068878,0.675000,4.900000,0.054821,2.653061
147,"(22551, 22554)",(22556),0.104592,0.168367,0.068878,0.658537,3.911308,0.051268,2.435496


# Excersice1

In [16]:
unique_stocks = basket_df.drop_duplicates(subset=["StockCode"])
def name_replacement (x) : 
    items = list(x)
    new_names = []
    for item in items:
        row = unique_stocks.loc[unique_stocks['StockCode'].apply(lambda y: str(y) == str(item))]
        if not row["Description"].empty:
            new_names.append(list(row["Description"])[0])
        else:
            new_names.append(item)
            
    return tuple(new_names)

In [17]:
rules['antecedents'] = rules['antecedents'].apply(name_replacement)
print("antecedents column mapped")

antecedents column mapped


In [18]:
rules['consequents'] = rules['consequents'].apply(name_replacement)
print("consequents column mapped")

consequents column mapped


In [19]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(JUMBO BAG WOODLAND ANIMALS,)","(POSTAGE,)",0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf
1,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...","(POSTAGE,)",0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf
2,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...","(POSTAGE,)",0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf
3,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...","(SET/6 RED SPOTTY PAPER PLATES,)",0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959
4,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...","(SET/6 RED SPOTTY PAPER CUPS,)",0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796


In [20]:
rules.loc[condition0 & condition1 & condition2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...","(SET/6 RED SPOTTY PAPER PLATES,)",0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
4,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...","(SET/6 RED SPOTTY PAPER CUPS,)",0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
53,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...","(ALARM CLOCK BAKELIKE RED,)",0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
54,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...","(ALARM CLOCK BAKELIKE GREEN,)",0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194
81,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...","(SET/20 RED RETROSPOT PAPER NAPKINS,)",0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
84,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...","(ALARM CLOCK BAKELIKE PINK,)",0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456
109,"(PLASTERS IN TIN SPACEBOY, PLASTERS IN TIN CIR...","(PLASTERS IN TIN WOODLAND ANIMALS,)",0.089286,0.170918,0.068878,0.771429,4.513433,0.053617,3.627232
143,"(PLASTERS IN TIN WOODLAND ANIMALS, PLASTERS IN...","(PLASTERS IN TIN SPACEBOY,)",0.102041,0.137755,0.068878,0.675000,4.900000,0.054821,2.653061
147,"(PLASTERS IN TIN SPACEBOY, PLASTERS IN TIN WOO...","(PLASTERS IN TIN CIRCUS PARADE,)",0.104592,0.168367,0.068878,0.658537,3.911308,0.051268,2.435496


# Excersize2

In [21]:
swit_basket = basket_df.loc[basket_df['Country'] == 'Switzerland']
swit_basket

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
5320,536858,22326,ROUND SNACK BOXES SET OF4 WOODLAND,30,2010-12-03 10:36:00,2.95,13520.0,Switzerland
5321,536858,22554,PLASTERS IN TIN WOODLAND ANIMALS,36,2010-12-03 10:36:00,1.65,13520.0,Switzerland
5322,536858,21731,RED TOADSTOOL LED NIGHT LIGHT,24,2010-12-03 10:36:00,1.65,13520.0,Switzerland
5323,536858,20677,PINK POLKADOT BOWL,16,2010-12-03 10:36:00,1.25,13520.0,Switzerland
5324,536858,20750,RED RETROSPOT MINI CASES,2,2010-12-03 10:36:00,7.95,13520.0,Switzerland
...,...,...,...,...,...,...,...,...
508666,579259,22915,ASSORTED BOTTLE TOP MAGNETS,12,2011-11-29 08:40:00,0.42,12451.0,Switzerland
508667,579259,23178,JAM CLOCK MAGNET,6,2011-11-29 08:40:00,2.89,12451.0,Switzerland
508668,579259,23180,MUM'S KITCHEN CLOCK,4,2011-11-29 08:40:00,4.95,12451.0,Switzerland
508669,579259,23079,TOADSTOOL BEDSIDE LIGHT,2,2011-11-29 08:40:00,8.95,12451.0,Switzerland


In [22]:
sw_basket, sw_trans_nom = trans_basket_provider(swit_basket)

In [23]:
sw_binary_df = binary_df_provider(sw_basket, sw_trans_nom)
sw_binary_df.head()

,10002,10125,10133,15056BL,15056P,15058A,15058B,15060B,16008,16012,...,85174,85175,85176,85178,85206A,85227,85232D,90108,90114,POST
536858,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
539488,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
541518,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
541569,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
542586,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True


In [24]:
sw_frequencies_items = apriori(sw_binary_df, min_support= 0.07, use_colnames= True)

sw_rules = association_rules(sw_frequencies_items, metric= "lift", min_threshold= 1)

sw_rules = sw_rules.sort_values(['confidence', 'lift'], ascending= [False, False])

sw_rules.reset_index(inplace= True, drop= True)

sw_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(22544),(22539),0.074074,0.074074,0.074074,1.0,13.5,0.068587,inf
1,(22539),(22544),0.074074,0.074074,0.074074,1.0,13.5,0.068587,inf
2,(23292),(23290),0.074074,0.074074,0.074074,1.0,13.5,0.068587,inf
3,(23290),(23292),0.074074,0.074074,0.074074,1.0,13.5,0.068587,inf
4,"(22551, 20724)",(20717),0.074074,0.074074,0.074074,1.0,13.5,0.068587,inf


In [25]:
cond0 = sw_rules.antecedents.apply(lambda x: False if 'POST' in x else True)

cond1 = sw_rules.consequents.apply(lambda x: False if 'POST' in x else True)

cond2 = sw_rules['antecedents'].apply(lambda x: len(x) >= 2)

sw_rules.loc[cond0 & cond1 & cond2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(22551, 20724)",(20717),0.074074,0.074074,0.074074,1.000000,13.500000,0.068587,inf
6,"(23388, 22554)",(23389),0.074074,0.074074,0.074074,1.000000,13.500000,0.068587,inf
8,"(22554, 22551, 20724)",(20717),0.074074,0.074074,0.074074,1.000000,13.500000,0.068587,inf
9,"(20717, 22554)","(22551, 20724)",0.074074,0.074074,0.074074,1.000000,13.500000,0.068587,inf
10,"(22551, 20724)","(20717, 22554)",0.074074,0.074074,0.074074,1.000000,13.500000,0.068587,inf
...,...,...,...,...,...,...,...,...,...
2792,"(22551, 22554)",(23199),0.240741,0.129630,0.074074,0.307692,2.373626,0.042867,1.257202
2793,"(22551, 22554)","(22326, 22029)",0.240741,0.129630,0.074074,0.307692,2.373626,0.042867,1.257202
2794,"(22551, 22554)",(20685),0.240741,0.166667,0.074074,0.307692,1.846154,0.033951,1.203704
2795,"(22551, 22554)",(22029),0.240741,0.203704,0.074074,0.307692,1.510490,0.025034,1.150206


In [26]:
sw_rules.corrwith(rules, numeric_only=True)

antecedent support   -0.085105
consequent support   -0.644130
support              -0.110587
confidence                 NaN
lift                  0.178165
leverage              0.070309
conviction                 NaN
dtype: float64